In [4]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from gensim import models
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
import os


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
eat_info = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/광진구_카페_detail.csv")
eat_review = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/광진구_카페_review.csv")

In [6]:
eat_info

,Unnamed: 0,가게이름,가게 주소,음식 종류,가격 범위,리뷰개수,평점,lat,lng
0,1,뚝방길 홍차가게,서울특별시 광진구 자양강변길 277,카페 / 디저트,만원 미만,30,4.5,37.529946,127.073843
1,2,스미다,서울특별시 광진구 천호대로109길 18 1F,카페 / 디저트,만원 미만,5,4.2,37.557517,127.081245
2,3,Daughter,서울특별시 광진구 아차산로31길 40,카페 / 디저트,만원 미만,11,4.2,37.542645,127.070165
3,4,최가커피회관,서울특별시 광진구 능동로13길 30 B1,카페 / 디저트,만원 미만,7,4.2,37.543360,127.070580
4,5,Index,서울특별시 광진구 아차산로 200 3F,카페 / 디저트,만원 미만,34,4.1,37.540869,127.066014
...,...,...,...,...,...,...,...,...,...
150,151,메종드쁘띠푸르,서울특별시 광진구 아차산로 200,카페 / 디저트,만원 미만,3,NaN,37.540869,127.066014
151,152,더파빌리온,서울특별시 광진구 워커힐로 177 그랜드 워커힐 본관 1F,카페 / 디저트,만원-2만원,13,3.7,37.555213,127.110890
152,153,줄리앤몰리,서울특별시 광진구 뚝섬로59길 67 1F,카페 / 디저트,만원-2만원,4,NaN,37.534731,127.082997
153,154,케이크를부탁해,서울특별시 광진구 능동로 137,카페 / 디저트,만원-2만원,5,NaN,37.543065,127.071800


In [7]:
eat_review

,Unnamed: 0,가게이름,리뷰아이디,리뷰,맛평가
0,1,스타벅스,양양,"먹은 거: 오늘의커피(3,800원) 조각케익친절함.매장 청결함.공부하는 사람과 대화...",괜찮다
1,2,카페샛길,달리자크롱,"건대에서 조용히 얘기하면서 좋은 시간 보낼 수 있는 카페, 샛길자주 가도 리뷰 한번...",맛있다
2,3,투썸플레이스,muk_so.o,여러번 먹고 쓰는 리뷰 43스트로베리 요거 프라페/ 딸기 프라페에 요거트가 들어가고...,괜찮다
3,4,투썸플레이스,뒤빈,시간이 잠깐 떠서 + 이대 근처에 투썸이 없었는데 생겼길래 가봤어요. 신규 매장이라...,괜찮다
4,5,카페구들,심아호,아주 만족스러웠습니다!가게는 작은데 사장님두 넘넘 친절하시고 분위기도 따숩고 좋았어...,맛있다
...,...,...,...,...,...
1194,1195,스타벅스,뽐뭉,층이 여러개인 매장이지만 매장내부에서 층간 연결이 되어있지않아 주문후 다른층으로 이...,별로
1195,1196,탭플레이,수연,생딸기 모히또 맛있네여! 애플민트를 직접 빻아서 만든다고 되어있어요. 모히또 전문점...,맛있다
1196,1197,투썸플레이스,먹방꼬북,쾌적하고 좋아요,괜찮다
1197,1198,카페모든날,슈빙,친구 강아지 데려갈 수 있는 카페 찾아서 갔던 곳.시그니처 아인슈페너 마셨는데 많이...,괜찮다


In [8]:
eat_info.columns

Index(['Unnamed: 0', '가게이름', '가게 주소', '음식 종류', '가격 범위', '리뷰개수', '평점', 'lat',
       'lng'],
      dtype='object')

In [9]:
eat_info.rename(columns={'가게이름': 'name'}, inplace =True)
eat_info.rename(columns={'리뷰': 'review'}, inplace =True)
eat_info.rename(columns={'평점': 'rating'}, inplace =True)
eat_info.rename(columns={'리뷰개수': 'rvw_cnt'}, inplace =True)
eat_info.rename(columns={'가게 주소': 'address'}, inplace =True)
eat_info.rename(columns={'음식 종류': 'category'}, inplace =True)
eat_info.rename(columns={'가격 범위': 'price'}, inplace =True)




eat_review.rename(columns={'가게이름': 'res_name'}, inplace =True)
eat_review.rename(columns={'리뷰아이디':  'user_name'}, inplace =True)
eat_review.rename(columns={'리뷰':  'review'}, inplace =True)
eat_review.rename(columns={'맛평가':  'rating'}, inplace =True)

eat_info.drop(['Unnamed: 0'],axis=1, inplace=True)
eat_review.drop(['Unnamed: 0'],axis=1, inplace=True)



In [10]:
eat_info.columns

Index(['name', 'address', 'category', 'price', 'rvw_cnt', 'rating', 'lat',
       'lng'],
      dtype='object')

In [11]:
eat_info = eat_info[eat_info['name'].notnull()]
eat_review = eat_review[eat_review['user_name'].notnull()]
eat_info.reset_index(drop=True,inplace=True)
eat_review.reset_index(drop=True,inplace=True)

In [12]:
#delete overlapped data
for id in eat_info.index:
    if list(eat_info['name']).count(eat_info.loc[id,'name'])>1:
        eat_info.drop(id,inplace=True)

In [13]:
#label encoding, Place_name -> p_id, User_name -> u_id
le = LabelEncoder()
le.fit(eat_info['name'])
eat_info['p_id'] = le.transform(eat_info['name'])
eat_review['p_id'] = le.transform(eat_review['res_name'])

user_le = LabelEncoder()
user_le.fit(eat_review['user_name'])
eat_review['u_id'] = user_le.transform(eat_review['user_name'])

In [14]:
# delete punctuation marks
def delete_punctuation_marks_review(review):
    review = review.replace('.', '. ');
    review = review.replace('!', '! ');
    review = review.replace('(', '( ');
    review = review.replace(')', ') ');
    review = review.replace('^', '');
    review = review.replace('*', '');
    review = review.replace('-', ' ');
    review = review.replace('\n', '. ');
    review = review.replace('\"', '. ');
    return review

In [15]:
eat_info.head()


,name,address,category,price,rvw_cnt,rating,lat,lng,p_id
0,뚝방길 홍차가게,서울특별시 광진구 자양강변길 277,카페 / 디저트,만원 미만,30,4.5,37.529946,127.073843,20
1,스미다,서울특별시 광진구 천호대로109길 18 1F,카페 / 디저트,만원 미만,5,4.2,37.557517,127.081245,61
2,Daughter,서울특별시 광진구 아차산로31길 40,카페 / 디저트,만원 미만,11,4.2,37.542645,127.070165,3
3,최가커피회관,서울특별시 광진구 능동로13길 30 B1,카페 / 디저트,만원 미만,7,4.2,37.543360,127.070580,99
4,Index,서울특별시 광진구 아차산로 200 3F,카페 / 디저트,만원 미만,34,4.1,37.540869,127.066014,7


In [16]:
eat_review.head()


,res_name,user_name,review,rating,p_id,u_id
0,스타벅스,양양,"먹은 거: 오늘의커피(3,800원) 조각케익친절함.매장 청결함.공부하는 사람과 대화...",괜찮다,63,416
1,카페샛길,달리자크롱,"건대에서 조용히 얘기하면서 좋은 시간 보낼 수 있는 카페, 샛길자주 가도 리뷰 한번...",맛있다,110,252
2,투썸플레이스,muk_so.o,여러번 먹고 쓰는 리뷰 43스트로베리 요거 프라페/ 딸기 프라페에 요거트가 들어가고...,괜찮다,132,148
3,투썸플레이스,뒤빈,시간이 잠깐 떠서 + 이대 근처에 투썸이 없었는데 생겼길래 가봤어요. 신규 매장이라...,괜찮다,132,266
4,카페구들,심아호,아주 만족스러웠습니다!가게는 작은데 사장님두 넘넘 친절하시고 분위기도 따숩고 좋았어...,맛있다,103,406


In [17]:
eat_info['reviews'] = ''


In [18]:
# from requests import Request
# from requests import Session


# def KOtoEN(review):
#     s=Session()

#     headers ={
#         "X-Naver-Client-Id": "NsHid2ZsKcP5QclyL66L",
#         "X-Naver-Client-Secret": "OPb4L85YOo",
#     }

#     payload ={
#         'source':'ko',
#         'target':'en',
#         'text': review,
#     }

#     url ="https://openapi.naver.com/v1/papago/n2mt"
#     # "https://openapi.naver.com/v1/language/translate"

#     req =Request('POST', url, data=payload, headers=headers)
#     prepped =req.prepare()
#     res =s.send(prepped)
#     print(res.json())
#     # result =res.json()['message']['result']['translatedText']
#     return result

In [19]:
review_1=eat_review['review'][1]
review_1

'건대에서 조용히 얘기하면서 좋은 시간 보낼 수 있는 카페, 샛길자주 가도 리뷰 한번 쓴 곳은 다시 안올리게 되는 것 같다. 귀찮고 새로운 내용도 딱히 없기 때문에 풍부하게 글쓰기가 어렵기 때문이다.근데 이곳은 디저트가 넘나 맛있고! 사장님도 짱 친절해서 장사가 더 잘 됐으면 좋겠다는 바람으로 또 올리고 싶어졌다:) 처음으로 두번 쓰는 리뷰다!여긴 빈티지 소품들이 많아 갬성 돋는다. 샛길에 있어서 사람들이 잘 모르는지 사람이 별로 없고 조용해서 더 좋다. 노래도, 분위기도 얘기하기 좋다.사실 티라미수 케이크 먹으려고 갔는데 찾는 사람이 없어 이젠 안나온다고 한다 ㅠ_ㅠ 내가 먹어 본 티라미수 중에 제일 맛있었는데 너무 아쉽다..• 바스크치즈케이크 5500원 그래도 이번에 시킨 케이크 역시 만족스러웠다. 여긴 디저트 맛집이다! 치즈가 느끼하지도 않고 부드러웠다. 개인적으로 치즈케이크를 많이 좋아해서 맛있게 먹었다. 빵은 폭신하고 크림치즈가 들어있는 중간은 부드럽고 따뜻했다.• 바닐라 라떼 5천원, 헤이즐넛 라떼 5천원 우유가 고소하고 부드러웠다. 무난하게 먹기 좋은 커피였다. 요즘 아메리카노만 먹다가 단 음료 먹으니깐 더 맛있게 느껴졌다 ㅋ_ㅋ카페 분위기가 좋아 또 찾아갈거다. 재방문예정 :-)'

In [20]:
# KOtoEN(review_1)

In [21]:
# ts = Translator()
# eat_review['review_eng'] = ''

# #translating review
# for id in tqdm(eat_review.index):
#     review = eat_review.loc[id,'review']
    
#     if type(review)==float:
#         continue
        
#     review_pre = delete_punctuation_marks_review(review)
    
#     try:
#         review_ts = KOtoEN(review_pre)
#     except:
#         review_ts = ''

#     print(review_ts)
    
#     eat_review.loc[id,'review_eng'] = review_ts

In [22]:
# ts = Translator()

# ts.translate(review_1, source='ko', target='en', verbose=False)

In [23]:
# ts = Translator()
# eat_review['review_eng'] = ''

# #translating review
# for id in tqdm(eat_review.index):
#     review = eat_review.loc[id,'review']
    
#     if type(review)==float:
#         continue
        
#     review_pre = delete_punctuation_marks_review(review)
    
#     try:
#         review_ts = ts.translate(review_pre, source='ko', target='en', verbose=False)
#     except:
#         review_ts = ''
    
#     eat_review.loc[id,'review_eng'] = review_ts

In [24]:
for id in eat_info['p_id']:
    review_all = ''
    
    for review in eat_review[eat_review['p_id']==id]['review']:
        review_all+=review +'. '
        
    eat_info.loc[eat_info['p_id']==id,'reviews'] = review_all

In [25]:
eat_info.head()


,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews
0,뚝방길 홍차가게,서울특별시 광진구 자양강변길 277,카페 / 디저트,만원 미만,30,4.5,37.529946,127.073843,20,# 뚝방길 크림티 쉬는 날 방문짧은 휴가를 여유롭고 사랑스럽게 만들어준 곳. 뚝방길...
1,스미다,서울특별시 광진구 천호대로109길 18 1F,카페 / 디저트,만원 미만,5,4.2,37.557517,127.081245,61,날이 많이 더워져서 젤라또 먹기 좋은 날씨 같다. 어린이 대공원에서 가깝긴 하지만 ...
2,Daughter,서울특별시 광진구 아차산로31길 40,카페 / 디저트,만원 미만,11,4.2,37.542645,127.070165,3,"커피와 술이 다 있는 곳을 찾다가 들어간 곳-!! 와인이 있길래 시켜서 먹었는데, ..."
3,최가커피회관,서울특별시 광진구 능동로13길 30 B1,카페 / 디저트,만원 미만,7,4.2,37.543360,127.070580,99,최가커피 본점을 방문한적이 있었는데 그때 좀 가게가 낡았던....기억이 있어서 분점...
4,Index,서울특별시 광진구 아차산로 200 3F,카페 / 디저트,만원 미만,34,4.1,37.540869,127.066014,7,꼭 한번 와보고 싶었다. 여기서 커피와 독서를 즐겨보고 싶었다. 결과적으로 나는 대...


In [26]:
eat_info['price'].unique()

array(['만원 미만', '만원-2만원'], dtype=object)

In [27]:
eat_info['price'].replace('만원 미만',5000, inplace=True)
eat_info['price'].replace('만원-2만원',15000, inplace=True)
eat_info['price'].replace('2만원-3만원',25000, inplace=True)
eat_info['price'].replace('3만원-4만원',35000, inplace=True)
eat_info['price'].replace('4만원 이상',50000, inplace=True)

In [28]:
eat_review.head()


,res_name,user_name,review,rating,p_id,u_id
0,스타벅스,양양,"먹은 거: 오늘의커피(3,800원) 조각케익친절함.매장 청결함.공부하는 사람과 대화...",괜찮다,63,416
1,카페샛길,달리자크롱,"건대에서 조용히 얘기하면서 좋은 시간 보낼 수 있는 카페, 샛길자주 가도 리뷰 한번...",맛있다,110,252
2,투썸플레이스,muk_so.o,여러번 먹고 쓰는 리뷰 43스트로베리 요거 프라페/ 딸기 프라페에 요거트가 들어가고...,괜찮다,132,148
3,투썸플레이스,뒤빈,시간이 잠깐 떠서 + 이대 근처에 투썸이 없었는데 생겼길래 가봤어요. 신규 매장이라...,괜찮다,132,266
4,카페구들,심아호,아주 만족스러웠습니다!가게는 작은데 사장님두 넘넘 친절하시고 분위기도 따숩고 좋았어...,맛있다,103,406


In [29]:
eat_review['rating'].unique()

array(['괜찮다', '맛있다', '별로'], dtype=object)

In [30]:
eat_review['rating'].replace('맛있다',5, inplace=True)
eat_review['rating'].replace('괜찮다',3, inplace=True)
eat_review['rating'].replace('별로',0, inplace=True)

In [32]:
eat_info

,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews
0,뚝방길 홍차가게,서울특별시 광진구 자양강변길 277,카페 / 디저트,5000,30,4.5,37.529946,127.073843,20,# 뚝방길 크림티 쉬는 날 방문짧은 휴가를 여유롭고 사랑스럽게 만들어준 곳. 뚝방길...
1,스미다,서울특별시 광진구 천호대로109길 18 1F,카페 / 디저트,5000,5,4.2,37.557517,127.081245,61,날이 많이 더워져서 젤라또 먹기 좋은 날씨 같다. 어린이 대공원에서 가깝긴 하지만 ...
2,Daughter,서울특별시 광진구 아차산로31길 40,카페 / 디저트,5000,11,4.2,37.542645,127.070165,3,"커피와 술이 다 있는 곳을 찾다가 들어간 곳-!! 와인이 있길래 시켜서 먹었는데, ..."
3,최가커피회관,서울특별시 광진구 능동로13길 30 B1,카페 / 디저트,5000,7,4.2,37.543360,127.070580,99,최가커피 본점을 방문한적이 있었는데 그때 좀 가게가 낡았던....기억이 있어서 분점...
4,Index,서울특별시 광진구 아차산로 200 3F,카페 / 디저트,5000,34,4.1,37.540869,127.066014,7,꼭 한번 와보고 싶었다. 여기서 커피와 독서를 즐겨보고 싶었다. 결과적으로 나는 대...
...,...,...,...,...,...,...,...,...,...,...
150,메종드쁘띠푸르,서울특별시 광진구 아차산로 200,카페 / 디저트,5000,3,NaN,37.540869,127.066014,38,빵 종류도 많고 빵은 괜찮아요. 벋 테이블 세 개만 내부에 있고 다 외부에요. 겨울...
151,더파빌리온,서울특별시 광진구 워커힐로 177 그랜드 워커힐 본관 1F,카페 / 디저트,15000,13,3.7,37.555213,127.110890,18,피자힐에서 그렇게 거나하게 먹어놓고 디저트가 들어갈 배가 아직 남아있다는게 증맬 놀...
152,줄리앤몰리,서울특별시 광진구 뚝섬로59길 67 1F,카페 / 디저트,15000,4,NaN,37.534731,127.082997,91,"선물하기좋은 귀염뽀짝마카롱,예약없이도 구매가능한 당일레터링케이크,매일 갈수밖에 없는..."
153,케이크를부탁해,서울특별시 광진구 능동로 137,카페 / 디저트,15000,5,NaN,37.543065,127.071800,125,"가격이 워낙 저렴하다보니 특별히 엄청 맛있는 곳은 아니지만, 부담스럽지 않은 가격에..."


In [33]:
eat_review

,res_name,user_name,review,rating,p_id,u_id
0,스타벅스,양양,"먹은 거: 오늘의커피(3,800원) 조각케익친절함.매장 청결함.공부하는 사람과 대화...",3,63,416
1,카페샛길,달리자크롱,"건대에서 조용히 얘기하면서 좋은 시간 보낼 수 있는 카페, 샛길자주 가도 리뷰 한번...",5,110,252
2,투썸플레이스,muk_so.o,여러번 먹고 쓰는 리뷰 43스트로베리 요거 프라페/ 딸기 프라페에 요거트가 들어가고...,3,132,148
3,투썸플레이스,뒤빈,시간이 잠깐 떠서 + 이대 근처에 투썸이 없었는데 생겼길래 가봤어요. 신규 매장이라...,3,132,266
4,카페구들,심아호,아주 만족스러웠습니다!가게는 작은데 사장님두 넘넘 친절하시고 분위기도 따숩고 좋았어...,5,103,406
...,...,...,...,...,...,...
1194,스타벅스,뽐뭉,층이 여러개인 매장이지만 매장내부에서 층간 연결이 되어있지않아 주문후 다른층으로 이...,0,63,367
1195,탭플레이,수연,생딸기 모히또 맛있네여! 애플민트를 직접 빻아서 만든다고 되어있어요. 모히또 전문점...,5,129,390
1196,투썸플레이스,먹방꼬북,쾌적하고 좋아요,3,132,317
1197,카페모든날,슈빙,친구 강아지 데려갈 수 있는 카페 찾아서 갔던 곳.시그니처 아인슈페너 마셨는데 많이...,3,109,394


In [35]:
eat_info.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_cafe_info.csv', index=False)
eat_review.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_cafe_review.csv', index=False)